### 神经网络
* 通过 torch.nn 包来构建
* 神经网络是基于自动梯度 (autograd)来定义一些模型
* 一个 nn.Module 包括层和一个方法 
  * forward(input) 它会返回输出(output)
#### 典型的神经网络训练过程
* 定义一个包含可训练参数的神经网络
* 迭代整个输入
* 通过神经网络了处理输入
* 计算损失
* 反向传播梯度到神经网络的参数
* 更新网络的参数

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

#### 定义神经网络
##### nn.Conv1d
* 用于文本数据，只对宽度进行卷积，对高度不卷积
  * 通常，输入大小为word_embedding_dim * max_length，其中，word_embedding_dim为词向量的维度，max_length为句子的最大长度
  * 卷积核窗口在句子长度的方向上滑动，进行卷积操作
* 主要参数说明
  * in_channels：在文本应用中，即为词向量的维度
  * out_channels：卷积产生的通道数，有多少个out_channels，就需要多少个一维卷积（也就是卷积核的数量）
  * kernel_size：卷积核的尺寸；卷积核的第二个维度由in_channels决定，所以实际上卷积核的大小为kernel_size * in_channels
  * padding：对输入的每一条边，补充0的层数

##### nn.Conv2d
* 对由多个输入平面组成的输入信号进行二维卷积
* 主要参数说明：
  * in_channels —— 输入的channels数
  * out_channels —— 输出的channels数
  * kernel_size ——卷积核的尺寸，可以是方形卷积核、也可以不是
  * stride —— 步长，用来控制卷积核移动间隔
  * padding ——输入边沿扩边操作
  * padding_mode ——扩边的方式
  * bias ——是否使用偏置(即out = wx+b中的b)
  * dilation —— 这个参数简单说，设定了取数之间的间隔
  * groups —— 进行分组卷积的组数
    * controls the connections between inputs and poutputs
    * in_channels and out_channels must both divisible by groups
##### MaxPool2d
* 主要参数
  * kernel_size ：表示做最大池化的窗口大小，可以是单个值，也可以是tuple元组
    * 滑动窗口，非卷积核，大小由自己制定
    * 输入单值3 即3×3，输入元组（3，2）即3×2
    * 取该窗口覆盖元素中的最大值
  * stride ：步长，可以是单个值，也可以是tuple元组
    * 确定窗口如何滑动
    * 默认为池化窗口大小
  * padding ：填充，可以是单个值，也可以是tuple元组
  * ilation ：控制窗口中元素步幅
  * return_indices ：布尔类型，返回最大值位置索引
  * ceil_mode ：布尔类型，为True，用向上取整的方法，计算输出形状；默认是向下取整

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()  #继承父类的init方法
        # input: 1 channel 图像； output：6 channels ；5 ×5 卷积核
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation（仿射函数）反映了一种从 k 维到 m 维的空间映射关系
        # y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # in_feature,out_feature,bias
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # max pooling over a (2,2) window
        print("输入形状：",x.size())
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        print("第一层卷积网络输出形状：", x.size())
        # if the side is a suare, you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        print("第二层卷积网络输出形状1：", x.size())
        x = x.view(-1, self.num_flat_features(x))
        print("第二层卷积网络输出形状2：", x.size())
        x = F.relu(self.fc1(x))
        print("全连接层1输出形状：", x.size())
        x = F.relu(self.fc2(x))
        print("全连接层2输出形状：", x.size())
        x = self.fc3(x)
        print("全连接层3输出形状：", x.size())
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


#### 处理输入，调用反向传播

In [4]:
net = Net()
print(net)

#返回模型的可训练参数 net.parameters()
params = list(net.parameters())
print(len(params))
print(params[0].size())

# 尝试随机生成一个32×32的输入
input = torch.randn(1, 1, 32, 32)
output = net(input)
print(output)

# 将所有参数梯度缓存器置零，用随机的梯度来反向传播
net.zero_grad()
output.backward(torch.randn(1, 10)) # 随机梯度


Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
10
torch.Size([6, 1, 5, 5])
输入形状： torch.Size([1, 1, 32, 32])
第一层卷积网络输出形状： torch.Size([1, 6, 14, 14])
第二层卷积网络输出形状1： torch.Size([1, 16, 5, 5])
第二层卷积网络输出形状2： torch.Size([1, 400])
全连接层1输出形状： torch.Size([1, 120])
全连接层2输出形状： torch.Size([1, 84])
全连接层3输出形状： torch.Size([1, 10])
tensor([[ 0.1051, -0.0245,  0.0836, -0.0030,  0.0028,  0.0225, -0.1032, -0.0201,
          0.0256, -0.0133]], grad_fn=<AddmmBackward0>)


#### 计算损失值
* 需要模型输出和目标，然后计算一个值来评估输出距离目标由多远
* 不同的损失函数在nn包中
    * 简单的损失函数就是nn.MSELoss

In [69]:
output = net(input)

target = torch.randn(10)  #假设值
target = target.view(1, -1)  #make it the same shape as output
print('目标值的形状', target.size())
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

# 跟随以下步骤来反向传播
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

# 清空现存的梯度
# 调用loss.backward()
# 看conv1的偏置项在反向传播前后的变化
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

输入形状： torch.Size([1, 1, 32, 32])
第一层卷积网络输出形状： torch.Size([1, 6, 14, 14])
第二层卷积网络输出形状1： torch.Size([1, 16, 5, 5])
第二层卷积网络输出形状2： torch.Size([1, 400])
全连接层1输出形状： torch.Size([1, 120])
全连接层2输出形状： torch.Size([1, 84])
全连接层3输出形状： torch.Size([1, 10])
目标值的形状 torch.Size([1, 10])
tensor(0.4571, grad_fn=<MseLossBackward0>)
conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0011, -0.0055, -0.0038,  0.0003,  0.0041,  0.0002])


#### 更新神经网络的参数
* 最简单的更新规则：随机梯度下降
* weight=weight-learning_rate*gradient

In [70]:
learning_rate= 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)